# Hate Speech Detection using CNN

This notebook builds a Convolutional Neural Network to classify text as hate speech or not.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.utils import to_categorical


ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Load dataset
data = pd.read_csv('../data/hatespeech.csv')  # adjust path as needed
data.head()


In [ ]:
# Simple preprocessing
data.dropna(subset=['text'], inplace=True)
texts = data['text'].values
labels = data['label'].values

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = to_categorical(labels_encoded)

# Tokenize text
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, padding='post', maxlen=100)

# Split
X_train, X_test, y_train, y_test = train_test_split(padded, labels_encoded, test_size=0.2, random_state=42)


In [ ]:
# CNN Model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(labels_encoded.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
# Train model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)


In [ ]:
# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')


In [ ]:
# Predict function
def predict_hate(text):
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, padding='post', maxlen=100)
    pred = model.predict(pad)
    return label_encoder.classes_[np.argmax(pred)]

# Try it
predict_hate("I hate you and everything about you.")
